In [2]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime
import random 

In [3]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols

def get_crypto_syms():
   #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
   'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols

def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest

def scaler(row):
    scaler = MinMaxScaler(feature_range=(-6, 6))
    row = scaler.fit_transform(row)
    return row

def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    for i in range(12, data.shape[0]):
        orow = []
        crow = []
        hrow = []
        lrow = []
        vrow = []

        o1110 = data[i-1][0] - data[i-2][0]
        o1009 = data[i-2][0] - data[i-3][0]
        o0908 = data[i-3][0] - data[i-4][0]
        o0807 = data[i-4][0] - data[i-5][0]
        o0706 = data[i-5][0] - data[i-6][0]
        o6505 = data[i-6][0] - data[i-7][0]
        o0504 = data[i-7][0] - data[i-8][0]
        o0403 = data[i-8][0] - data[i-9][0]
        o0302 = data[i-9][0] - data[i-10][0]
        o0201 = data[i-10][0] - data[i-11][0]
        o0100 = data[i-11][0] - data[i-12][0]

        orow.append([o1110, o1009, o0908, o0807, o0706,
                    o6505, o0504, o0403, o0302, o0201, o0100])
        orow = np.array(orow, dtype="object")
        orow = orow.reshape(-1, 1)
        orow = scaler(orow)

        c1110 = data[i-1][3] - data[i-2][3]
        c1009 = data[i-2][3] - data[i-3][3]
        c0908 = data[i-3][3] - data[i-4][3]
        c0807 = data[i-4][3] - data[i-5][3]
        c0706 = data[i-5][3] - data[i-6][3]
        c6505 = data[i-6][3] - data[i-7][3]
        c0504 = data[i-7][3] - data[i-8][3]
        c0403 = data[i-8][3] - data[i-9][3]
        c0302 = data[i-9][3] - data[i-10][3]
        c0201 = data[i-10][3] - data[i-11][3]
        c0100 = data[i-11][3] - data[i-12][3]

        crow.append([c1110, c1009, c0908, c0807, c0706,
                    c6505, c0504, c0403, c0302, c0201, c0100])
        crow = np.array(crow, dtype="object")
        crow = crow.reshape(-1, 1)
        crow = scaler(crow)

        h1110 = data[i-1][1] - data[i-2][1]
        h1009 = data[i-2][1] - data[i-3][1]
        h0908 = data[i-3][1] - data[i-4][1]
        h0807 = data[i-4][1] - data[i-5][1]
        h0706 = data[i-5][1] - data[i-6][1]
        h6505 = data[i-6][1] - data[i-7][1]
        h0504 = data[i-7][1] - data[i-8][1]
        h0403 = data[i-8][1] - data[i-9][1]
        h0302 = data[i-9][1] - data[i-10][1]
        h0201 = data[i-10][1] - data[i-11][1]
        h0100 = data[i-11][1] - data[i-12][1]

        hrow.append([h1110, h1009, h0908, h0807, h0706,
                    h6505, h0504, h0403, h0302, h0201, h0100])
        hrow = np.array(hrow, dtype="object")
        hrow = hrow.reshape(-1, 1)
        hrow = scaler(hrow)

        l1100 = data[i-1][2] - data[i-2][2]
        l1009 = data[i-2][2] - data[i-3][2]
        l0908 = data[i-3][2] - data[i-4][2]
        l0807 = data[i-4][2] - data[i-5][2]
        l0706 = data[i-5][2] - data[i-6][2]
        l6505 = data[i-6][2] - data[i-7][2]
        l0504 = data[i-7][2] - data[i-8][2]
        l0403 = data[i-8][2] - data[i-9][2]
        l0302 = data[i-9][2] - data[i-10][2]
        l0201 = data[i-10][2] - data[i-11][2]
        l0100 = data[i-11][2] - data[i-12][2]

        lrow.append([l1100, l1009, l0908, l0807, l0706,
                    l6505, l0504, l0403, l0302, l0201, l0100])
        lrow = np.array(lrow, dtype="object")
        lrow = lrow.reshape(-1, 1)
        lrow = scaler(lrow)
        
        v1110 = data[i-1][5] - data[i-2][5]
        v1009 = data[i-2][5] - data[i-3][5]
        v0908 = data[i-3][5] - data[i-4][5]
        v0807 = data[i-4][5] - data[i-5][5]
        v0706 = data[i-5][5] - data[i-6][5]
        v6505 = data[i-6][5] - data[i-7][5]
        v0504 = data[i-7][5] - data[i-8][5]
        v0403 = data[i-8][5] - data[i-9][5]
        v0302 = data[i-9][5] - data[i-10][5]
        v0201 = data[i-10][5] - data[i-11][5]
        v0100 = data[i-11][5] - data[i-12][5]

        vrow.append([v1110, v1009, v0908, v0807, v0706, v6505, v0504, v0403, v0302, v0201, v0100])
        vrow = np.array(vrow, dtype="object")
        vrow = vrow.reshape(-1, 1)
        vrow = scaler(vrow)

        sugg = 0
        if data[i][3] > data[i][0]:
            sugg = 1

        orow = list(orow)
        crow = list(crow)
        hrow = list(hrow)
        lrow = list(lrow)
        vrow = list(vrow)

        row.append(orow + crow + hrow + lrow + vrow + [sugg])
    row = np.array(row, dtype="object")
    return row


In [4]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
              pass
           else:
               if data.shape[0] > 12:
                   unattached_dfs.append(pd.DataFrame(process(data),columns=clmns))
    symbols = []
    symbol = []
    data = []

    data = pd.concat(unattached_dfs)
    data = data.astype(float)
    data = data.astype({'suggestion':'int'})

    #xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,1000)}.csv")
    #return xTrain, xTest, yTrain, yTest,data

#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

pieces = 5
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays:
   get_data(symbols)
    
#xTrain, xTest, yTrain, yTest,data= get_data(symbols)
#data

(289543, 56)
(368497, 56)
(369626, 56)
(369985, 56)
(377795, 56)


In [5]:
data = pd.read_csv("597.csv")

In [6]:
data = pd.concat([data,pd.read_csv("4.csv")])

In [7]:
data = pd.concat([data,pd.read_csv("675.csv")])
data = pd.concat([data,pd.read_csv("730.csv")])
data = pd.concat([data,pd.read_csv("735.csv")])

In [8]:
data

,Unnamed: 0,Open 12-11,Open 11-10,Open 10-9,Open 9-8,Open 8-7,Open 7-6,Open 6-5,Open 5-4,Open 4-3,...,Volume 10-9,Volume 9-8,Volume 8-7,Volume 7-6,Volume 6-5,Volume 5-4,Volume 4-3,Volume 3-2,Volume 2-1,suggestion
0,0,-0.601479,-0.113644,-0.825993,-0.328920,-6.000000,1.854326,3.846320,-2.150909,6.000000,...,-1.986657,-3.452038,0.073734,-6.000000,-4.131429,3.656705,0.090830,6.000000,-1.546188,0
1,1,-1.153988,-0.601479,-0.113644,-0.825993,-0.328920,-6.000000,1.854326,3.846320,-2.150909,...,-5.211305,-1.986657,-3.452038,0.073734,-6.000000,-4.131429,3.656705,0.090830,6.000000,0
2,2,-6.000000,-1.112913,-0.563577,-0.078543,-0.786802,-0.292582,-5.931102,1.878129,3.858685,...,-1.005037,-5.019920,-1.012780,-2.833749,1.547585,-6.000000,-3.678001,6.000000,1.568830,1
3,3,0.702199,-6.000000,-0.051434,0.617219,1.207603,0.345510,0.947074,-5.916137,3.589265,...,0.454277,-1.005037,-5.019920,-1.012780,-2.833749,1.547585,-6.000000,-3.678001,6.000000,1
4,4,2.118897,0.702199,-6.000000,-0.051434,0.617219,1.207603,0.345510,0.947074,-5.916137,...,4.942953,3.214640,1.131207,-4.600759,1.120153,-1.479608,4.775535,-6.000000,-2.684929,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377790,950,6.000000,0.806906,-0.675615,-6.000000,-0.021077,-1.898926,-0.798738,-1.065174,-1.448021,...,-0.177410,-6.000000,0.778137,-2.163638,4.774612,-5.549590,2.100484,0.530775,-4.553398,0
377791,951,0.630576,6.000000,0.806906,-0.675615,-6.000000,-0.021077,-1.898926,-0.798738,-1.065174,...,-3.231413,-0.177410,-6.000000,0.778137,-2.163638,4.774612,-5.549590,2.100484,0.530775,0
377792,952,-2.079540,0.630576,6.000000,0.806906,-0.675615,-6.000000,-0.021077,-1.898926,-0.798738,...,6.000000,-3.231413,-0.177410,-6.000000,0.778137,-2.163638,4.774612,-5.549590,2.100484,0
377793,953,-0.547667,-2.079540,0.630576,6.000000,0.806906,-0.675615,-6.000000,-0.021077,-1.898926,...,-0.663097,6.000000,-3.231413,-0.177410,-6.000000,0.778137,-2.163638,4.774612,-5.549590,1


In [9]:
data = data.drop("Unnamed: 0",axis=1)
xTrain, xTest, yTrain, yTest= spliting(data)


(1597901, 55) (1597901,)
(177545, 55) (177545,)


In [10]:
model = Sequential()



model.add(Dense(2048, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2048)              114688    
                                                                 
 dense_1 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_3 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 1)                 513       
                                                                 
Total params: 4,837,377
Trainable params: 4,837,377
Non-

In [12]:
data.to_parquet("mom.parquet")

In [11]:
model.fit(xTrain,yTrain,epochs=50,batch_size=10000,validation_data=(xTest,yTest))

Epoch 1/50
  6/160 [>.............................] - ETA: 12:10 - loss: 1.0799 - accuracy: 0.4971

KeyboardInterrupt: ignored

In [ ]:
model.save("cht.h5")
from google.colab import drive
drive.mount("/content/drive")

NameError: ignored